# 07. SVM 문제 풀이 집중 연습

## 학습 목표
- SVM Hard/Soft Margin 문제 풀이 마스터
- Lagrangian, Dual 유도 연습
- KKT 조건 적용
- 족보 문제 + 추가 문제

**기반 개념**: 
- 03_svm_hard_margin.ipynb
- 04_svm_soft_margin.ipynb
- 05_lagrange_multipliers.ipynb

---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import seaborn as sns

sns.set_style('whitegrid')
np.random.seed(42)

## 문제 1: SVM Hard Margin 기초

### 문제

다음 2D 데이터 포인트가 주어졌을 때:

- Positive: $(1, 1)$, $(2, 2)$
- Negative: $(-1, -1)$, $(-2, -2)$

1. Decision boundary $w^T x + b = 0$를 기하학적으로 추정하시오.
2. Margin이 최대인 $w$와 $b$를 찾으시오.
3. Support vectors를 식별하시오.
4. Margin width를 계산하시오.

### 풀이

**Step 1**: 기하학적 직관

- 데이터가 원점을 중심으로 대칭
- Decision boundary는 원점을 지나야 함
- 45도 각도 (대각선)

**Step 2**: 수식 설정

대칭성을 이용하면 $w = [1, -1]^T$ 또는 $[-1, 1]^T$  (방향 중요X)

$w = [1, -1]^T$, $b = 0$으로 시도:

- Positive점들: $w^T x = 1 \cdot 1 + (-1) \cdot 1 = 0$ (경계 위)
- 실제로는 margin이  필요!

**Step 3**: Primal problem

$$\begin{align}
\min_{w, b} \quad & \frac{1}{2}||w||^2 \\
\text{subject to} \quad & y_i(w^T x_i + b) \geq 1, \quad i = 1, 2, 3, 4
\end{align}$$

**Step 4**: Scipy로 풀기

In [ ]:
# 문제 1 코드 풀이

# 데이터
X = np.array([[1, 1],
              [2, 2],
              [-1, -1],
              [-2, -2]])
y = np.array([1, 1, -1, -1])

n_samples = len(y)

# Dual problem
# min 1/2 α^T Q α - 1^T α
# s.t. Σ α_i y_i = 0, α_i >= 0

K = X @ X.T  # Gram matrix
Q = np.outer(y, y) * K

def dual_obj(alpha):
    return 0.5 * alpha @ Q @ alpha - np.sum(alpha)

def dual_grad(alpha):
    return Q @ alpha - np.ones(n_samples)

# Constraints
cons = {'type': 'eq', 'fun': lambda alpha: np.sum(alpha * y)}
bounds = [(0, None) for _ in range(n_samples)]

# Optimize
alpha_init = np.ones(n_samples) * 0.1
result = minimize(dual_obj, alpha_init, method='SLSQP', 
                 jac=dual_grad, bounds=bounds, constraints=cons)

alpha_opt = result.x

print("=== 문제 1 풀이 ===")
print(f"\nOptimal α: {alpha_opt}")

# w 복원
w_opt = np.sum(alpha_opt[:, np.newaxis] * y[:, np.newaxis] * X, axis=0)
print(f"\nw = {w_opt}")
print(f"||w|| = {np.linalg.norm(w_opt):.4f}")

# b 계산 (support vectors)
sv_indices = np.where(alpha_opt > 1e-5)[0]
print(f"\nSupport vector indices: {sv_indices}")
print(f"Support vectors:")
for idx in sv_indices:
    print(f"  x_{idx} = {X[idx]}, y_{idx} = {y[idx]}, α_{idx} = {alpha_opt[idx]:.4f}")

b_values = [y[i] - w_opt @ X[i] for i in sv_indices]
b_opt = np.mean(b_values)
print(f"\nb = {b_opt:.4f}")

# Margin
margin = 2 / np.linalg.norm(w_opt)
print(f"\nMargin width = 2/||w|| = {margin:.4f}")

# 시각화
plt.figure(figsize=(10, 8))

# 데이터
plt.scatter(X[y==1, 0], X[y==1, 1], c='red', marker='o', s=200, label='Positive', edgecolors='k', linewidths=2)
plt.scatter(X[y==-1, 0], X[y==-1, 1], c='blue', marker='s', s=200, label='Negative', edgecolors='k', linewidths=2)

# Support vectors 강조
for idx in sv_indices:
    plt.scatter(X[idx, 0], X[idx, 1], s=400, facecolors='none', 
               edgecolors='yellow', linewidths=4)

# Decision boundary
x_line = np.linspace(-3, 3, 100)
y_line = -(w_opt[0] * x_line + b_opt) / w_opt[1]
plt.plot(x_line, y_line, 'g-', linewidth=3, label='Decision boundary')

# Margins
y_margin_pos = -(w_opt[0] * x_line + b_opt - 1) / w_opt[1]
y_margin_neg = -(w_opt[0] * x_line + b_opt + 1) / w_opt[1]
plt.plot(x_line, y_margin_pos, 'r--', linewidth=2, label='Margin (+1)')
plt.plot(x_line, y_margin_neg, 'b--', linewidth=2, label='Margin (-1)')

plt.xlabel('$x_1$', fontsize=14)
plt.ylabel('$x_2$', fontsize=14)
plt.title('SVM Hard Margin - 문제 1', fontsize=16, fontweight='bold')
plt.legend(fontsize=12)
plt.grid(alpha=0.3)
plt.axis('equal')
plt.xlim(-3, 3)
plt.ylim(-3, 3)
plt.show()

## 문제 2: Dual Problem 유도 연습

### 문제

3개 데이터 포인트:
- $(1, 0), y = +1$
- $(0, 1), y = +1$
- $(-1, 0), y = -1$

**과제**:
1. Primal problem 작성
2. Lagrangian 구성
3. Dual problem 유도 (손으로!)
4. 코드로 검증

### 상세 풀이

**Step 1: Primal**

$$\begin{align}
\min_{w, b} \quad & \frac{1}{2}(w_1^2 + w_2^2) \\
\text{s.t.} \quad & w_1 + b \geq 1 \quad (x_1 = (1, 0), y_1 = 1) \\
& w_2 + b \geq 1 \quad (x_2 = (0, 1), y_2 = 1) \\
& -(-w_1 + b) \geq 1 \quad (x_3 = (-1, 0), y_3 = -1) \\
& \Rightarrow w_1 - b \geq 1
\end{align}$$

**Step 2: Lagrangian**

$$\mathcal{L} = \frac{1}{2}(w_1^2 + w_2^2) - \alpha_1(w_1 + b - 1) - \alpha_2(w_2 + b - 1) - \alpha_3(w_1 - b - 1)$$

**Step 3: KKT Stationarity**

$$\frac{\partial \mathcal{L}}{\partial w_1} = w_1 - \alpha_1 - \alpha_3 = 0 \quad \Rightarrow \quad w_1 = \alpha_1 + \alpha_3$$

$$\frac{\partial \mathcal{L}}{\partial w_2} = w_2 - \alpha_2 = 0 \quad \Rightarrow \quad w_2 = \alpha_2$$

$$\frac{\partial \mathcal{L}}{\partial b} = -\alpha_1 - \alpha_2 + \alpha_3 = 0 \quad \Rightarrow \quad \alpha_1 + \alpha_2 = \alpha_3$$

**Step 4: Dual (대입)**

(복잡한 계산... 코드로 확인!)

In [ ]:
# 문제 2 코드 검증

X2 = np.array([[1, 0],
               [0, 1],
               [-1, 0]])
y2 = np.array([1, 1, -1])

n2 = len(y2)
K2 = X2 @ X2.T
Q2 = np.outer(y2, y2) * K2

print("=== 문제 2 ===")
print(f"\nGram matrix K:")
print(K2)
print(f"\nQ = y y^T ⊙ K:")
print(Q2)

def dual_obj2(alpha):
    return 0.5 * alpha @ Q2 @ alpha - np.sum(alpha)

cons2 = {'type': 'eq', 'fun': lambda alpha: np.sum(alpha * y2)}
bounds2 = [(0, None) for _ in range(n2)]

result2 = minimize(dual_obj2, np.ones(n2)*0.1, method='SLSQP',
                  bounds=bounds2, constraints=cons2)

alpha2 = result2.x
print(f"\nOptimal α: {alpha2}")

# w, b 복원
w2 = np.sum(alpha2[:, np.newaxis] * y2[:, np.newaxis] * X2, axis=0)
print(f"\nw = {w2}")

sv_idx2 = np.where(alpha2 > 1e-5)[0]
b2_vals = [y2[i] - w2 @ X2[i] for i in sv_idx2]
b2 = np.mean(b2_vals)
print(f"b = {b2:.4f}")

# 검증
print(f"\n=== 검증 ===")
for i in range(n2):
    margin_val = y2[i] * (w2 @ X2[i] + b2)
    print(f"y_{i}(w^T x_{i} + b) = {margin_val:.4f} (>= 1)")

# 시각화
plt.figure(figsize=(10, 8))
plt.scatter(X2[y2==1, 0], X2[y2==1, 1], c='red', marker='o', s=200, label='Positive', edgecolors='k', linewidths=2)
plt.scatter(X2[y2==-1, 0], X2[y2==-1, 1], c='blue', marker='s', s=200, label='Negative', edgecolors='k', linewidths=2)

# Decision boundary
x_vals = np.linspace(-2, 2, 100)
y_vals = -(w2[0] * x_vals + b2) / w2[1]
plt.plot(x_vals, y_vals, 'g-', linewidth=3, label='Decision boundary')

plt.xlabel('$x_1$', fontsize=14)
plt.ylabel('$x_2$', fontsize=14)
plt.title('문제 2: 3-point SVM', fontsize=16, fontweight='bold')
plt.legend()
plt.grid(alpha=0.3)
plt.axis('equal')
plt.xlim(-2, 2)
plt.ylim(-2, 2)
plt.show()

## 문제 3: Soft Margin with Outlier

### 문제

데이터:
- Positive: $(2, 2)$, $(3, 3)$, **$(-1, -0.5)$** (outlier!)
- Negative: $(-2, -2)$, $(-3, -3)$

**C parameter**: $C = 0.1$, $C = 1$, $C = 100$로 각각 풀고 비교.

### 풀이 전략

1. Outlier 없이 Hard Margin으로 먼저 시도 (불가능 확인)
2. Soft Margin ($C$ 작음): Outlier 무시
3. Soft Margin ($C$ 큼): Outlier에 끌려감

In [ ]:
# 문제 3: Soft Margin 비교

X3 = np.array([[2, 2],
               [3, 3],
               [-1, -0.5],  # Outlier
               [-2, -2],
               [-3, -3]])
y3 = np.array([1, 1, 1, -1, -1])

C_values = [0.1, 1, 100]

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, C in enumerate(C_values):
    # Dual with box constraint
    n3 = len(y3)
    K3 = X3 @ X3.T
    Q3 = np.outer(y3, y3) * K3
    
    def dual_obj3(alpha):
        return 0.5 * alpha @ Q3 @ alpha - np.sum(alpha)
    
    cons3 = {'type': 'eq', 'fun': lambda alpha: np.sum(alpha * y3)}
    bounds3 = [(0, C) for _ in range(n3)]  # Box constraint!
    
    result3 = minimize(dual_obj3, np.ones(n3)*0.1, method='SLSQP',
                      bounds=bounds3, constraints=cons3)
    
    alpha3 = result3.x
    w3 = np.sum(alpha3[:, np.newaxis] * y3[:, np.newaxis] * X3, axis=0)
    
    # b 계산 (0 < α < C인 점들)
    sv_margin = np.where((alpha3 > 1e-5) & (alpha3 < C - 1e-5))[0]
    if len(sv_margin) > 0:
        b3 = np.mean([y3[i] - w3 @ X3[i] for i in sv_margin])
    else:
        # 모든 SV 사용
        sv_all = np.where(alpha3 > 1e-5)[0]
        b3 = np.mean([y3[i] - w3 @ X3[i] for i in sv_all])
    
    # 시각화
    ax = axes[idx]
    
    # 데이터
    ax.scatter(X3[y3==1, 0], X3[y3==1, 1], c='red', marker='o', s=150, edgecolors='k', linewidths=2)
    ax.scatter(X3[y3==-1, 0], X3[y3==-1, 1], c='blue', marker='s', s=150, edgecolors='k', linewidths=2)
    
    # Outlier 강조
    ax.scatter(X3[2, 0], X3[2, 1], c='red', marker='*', s=500, edgecolors='black', linewidths=3, label='Outlier')
    
    # Decision boundary
    x_line = np.linspace(-4, 4, 100)
    y_line = -(w3[0] * x_line + b3) / w3[1]
    ax.plot(x_line, y_line, 'g-', linewidth=3)
    
    margin_width = 2 / np.linalg.norm(w3)
    
    ax.set_xlabel('$x_1$')
    ax.set_ylabel('$x_2$')
    ax.set_title(f'C = {C}\nMargin = {margin_width:.2f}')
    ax.grid(alpha=0.3)
    ax.axis('equal')
    ax.set_xlim(-4, 4)
    ax.set_ylim(-4, 4)
    if idx == 0:
        ax.legend()
    
    # α 출력
    print(f"\n=== C = {C} ===")
    print(f"α = {alpha3}")
    print(f"w = {w3}")
    print(f"Margin = {margin_width:.4f}")

plt.tight_layout()
plt.show()

print("\n해석:")
print("- C 작음: Outlier 무시, 큰 margin")
print("- C 큼: Outlier에 끌려감, 작은 margin")

## 문제 4: KKT 조건 검증

### 문제

Soft Margin SVM의 KKT 조건을 직접 검증하시오.

**KKT 조건 (복습)**:

1. **Stationarity**: $w = \sum_i \alpha_i y_i x_i$, $\sum_i \alpha_i y_i = 0$
2. **Primal feasibility**: $y_i(w^T x_i + b) \geq 1 - \xi_i$, $\xi_i \geq 0$
3. **Dual feasibility**: $0 \leq \alpha_i \leq C$
4. **Complementary slackness**:
   - $\alpha_i [y_i(w^T x_i + b) - 1 + \xi_i] = 0$
   - $(C - \alpha_i) \xi_i = 0$

### 케이스 분석

- $\alpha_i = 0$: Non-SV, $\xi_i = 0$, margin 밖
- $0 < \alpha_i < C$: SV on margin, $\xi_i = 0$
- $\alpha_i = C$: SV inside margin or misclassified, $\xi_i > 0$

In [ ]:
# 문제 4: KKT 검증

# 문제 3의 C=1 케이스 사용
C = 1
n3 = len(y3)
K3 = X3 @ X3.T
Q3 = np.outer(y3, y3) * K3

def dual_obj3(alpha):
    return 0.5 * alpha @ Q3 @ alpha - np.sum(alpha)

cons3 = {'type': 'eq', 'fun': lambda alpha: np.sum(alpha * y3)}
bounds3 = [(0, C) for _ in range(n3)]

result3 = minimize(dual_obj3, np.ones(n3)*0.1, method='SLSQP',
                  bounds=bounds3, constraints=cons3)

alpha3 = result3.x
w3 = np.sum(alpha3[:, np.newaxis] * y3[:, np.newaxis] * X3, axis=0)

sv_margin = np.where((alpha3 > 1e-5) & (alpha3 < C - 1e-5))[0]
if len(sv_margin) > 0:
    b3 = np.mean([y3[i] - w3 @ X3[i] for i in sv_margin])
else:
    sv_all = np.where(alpha3 > 1e-5)[0]
    b3 = np.mean([y3[i] - w3 @ X3[i] for i in sv_all])

print("=== KKT 조건 검증 (C=1) ===")

# 1. Stationarity
w_check = np.sum(alpha3[:, np.newaxis] * y3[:, np.newaxis] * X3, axis=0)
print(f"\n1. Stationarity:")
print(f"   w = Σ α_i y_i x_i: {np.allclose(w3, w_check)}")
print(f"   Σ α_i y_i = {np.sum(alpha3 * y3):.10f} (should be 0)")

# 2. Dual feasibility
print(f"\n2. Dual feasibility (0 ≤ α ≤ C):")
for i in range(n3):
    print(f"   α_{i} = {alpha3[i]:.4f}, valid: {0 <= alpha3[i] <= C}")

# 3. Primal feasibility & Slack
print(f"\n3. Primal feasibility & Slack:")
for i in range(n3):
    margin_val = y3[i] * (w3 @ X3[i] + b3)
    xi = max(0, 1 - margin_val)  # Slack variable
    print(f"   Point {i}: y(wx+b) = {margin_val:.4f}, ξ = {xi:.4f}")
    
    # 케이스 분류
    if alpha3[i] < 1e-5:
        case = "Non-SV (α=0)"
    elif alpha3[i] < C - 1e-5:
        case = "SV on margin (0<α<C)"
    else:
        case = "SV inside/misclassified (α=C)"
    print(f"              {case}")

# 4. Complementary slackness
print(f"\n4. Complementary slackness:")
for i in range(n3):
    margin_val = y3[i] * (w3 @ X3[i] + b3)
    xi = max(0, 1 - margin_val)
    slack1 = alpha3[i] * (margin_val - 1 + xi)
    slack2 = (C - alpha3[i]) * xi
    print(f"   Point {i}:")
    print(f"      α[y(wx+b)-1+ξ] = {slack1:.10f} (should be 0)")
    print(f"      (C-α)ξ = {slack2:.10f} (should be 0)")

## 문제 5: 족보 종합 문제

### 문제 (기출 스타일)

다음 데이터에 대해 Hard Margin SVM을 적용하시오:

- $(2, 0)^T, y = +1$
- $(0, 2)^T, y = +1$
- $(-2, 0)^T, y = -1$

**과제**:
1. Dual problem을 **손으로** 완전히 유도하시오.
2. Optimal $\alpha$를 구하시오.
3. $w$와 $b$를 복원하시오.
4. Margin width를 계산하시오.
5. Support vectors를 모두 나열하시오.

### 상세 해답

**(생략 - 시간 관계상 코드로 검증)**

In [ ]:
# 문제 5 풀이

X5 = np.array([[2, 0],
               [0, 2],
               [-2, 0]])
y5 = np.array([1, 1, -1])

print("=== 족보 종합 문제 ===")
print(f"\n데이터:")
for i in range(len(y5)):
    print(f"  x_{i} = {X5[i]}, y_{i} = {y5[i]:+d}")

# Dual
n5 = len(y5)
K5 = X5 @ X5.T
Q5 = np.outer(y5, y5) * K5

print(f"\nGram matrix K:")
print(K5)

def dual_obj5(alpha):
    return 0.5 * alpha @ Q5 @ alpha - np.sum(alpha)

cons5 = {'type': 'eq', 'fun': lambda alpha: np.sum(alpha * y5)}
bounds5 = [(0, None) for _ in range(n5)]

result5 = minimize(dual_obj5, np.ones(n5)*0.1, method='SLSQP',
                  bounds=bounds5, constraints=cons5)

alpha5 = result5.x
print(f"\n1. Optimal α: {alpha5}")

w5 = np.sum(alpha5[:, np.newaxis] * y5[:, np.newaxis] * X5, axis=0)
print(f"\n2. w = Σ α_i y_i x_i = {w5}")

sv5 = np.where(alpha5 > 1e-5)[0]
b5 = np.mean([y5[i] - w5 @ X5[i] for i in sv5])
print(f"\n3. b = {b5:.4f}")

margin5 = 2 / np.linalg.norm(w5)
print(f"\n4. Margin width = 2/||w|| = {margin5:.4f}")

print(f"\n5. Support vectors (α > 0):")
for i in sv5:
    print(f"   x_{i} = {X5[i]}, y_{i} = {y5[i]:+d}, α_{i} = {alpha5[i]:.4f}")

# 시각화
plt.figure(figsize=(10, 8))
plt.scatter(X5[y5==1, 0], X5[y5==1, 1], c='red', marker='o', s=200, label='Positive', edgecolors='k', linewidths=2)
plt.scatter(X5[y5==-1, 0], X5[y5==-1, 1], c='blue', marker='s', s=200, label='Negative', edgecolors='k', linewidths=2)

# Support vectors
for i in sv5:
    plt.scatter(X5[i, 0], X5[i, 1], s=400, facecolors='none', 
               edgecolors='yellow', linewidths=4)

# Decision boundary
x_line = np.linspace(-3, 3, 100)
y_line = -(w5[0] * x_line + b5) / w5[1]
plt.plot(x_line, y_line, 'g-', linewidth=3, label='Decision boundary')

plt.xlabel('$x_1$', fontsize=14)
plt.ylabel('$x_2$', fontsize=14)
plt.title('족보 종합 문제 풀이', fontsize=16, fontweight='bold')
plt.legend(fontsize=12)
plt.grid(alpha=0.3)
plt.axis('equal')
plt.xlim(-3, 3)
plt.ylim(-3, 3)
plt.show()

## 요약 및 핵심 공식

### SVM 문제 풀이 단계

1. **데이터 확인**: $(x_i, y_i)$, $i = 1, \ldots, n$
2. **Gram matrix**: $K_{ij} = x_i^T x_j$
3. **Q matrix**: $Q_{ij} = y_i y_j K_{ij}$
4. **Dual 풀기**: $\min \frac{1}{2}\alpha^T Q \alpha - 1^T \alpha$
5. **w 복원**: $w = \sum_i \alpha_i y_i x_i$
6. **b 복원**: $b = y_k - w^T x_k$ (support vector $k$)
7. **검증**: $y_i(w^T x_i + b) \geq 1 - \xi_i$

### Hard vs Soft Margin

| 항목 | Hard | Soft |
|------|------|------|
| Constraint | $\alpha_i \geq 0$ | $0 \leq \alpha_i \leq C$ |
| 선형분리 | 필수 | 불필요 |
| Outlier | 민감 | 강건 |
| Slack | $\xi_i = 0$ | $\xi_i \geq 0$ |

### KKT 조건 체크리스트

- [ ] Stationarity
- [ ] Primal feasibility
- [ ] Dual feasibility
- [ ] Complementary slackness

### 다음 단계

- Kernel SVM
- Non-linear classification
- SMO algorithm